# Pylops-distributed - Solvers

In this notebook we investigate the use of scipy solvers with distributed operators. 
We will also create a simple CG solver to compare the results and performance

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import skfmm
import dask.array as da
import pylops
import pylops_distributed

from scipy.sparse.linalg.interface import MatrixLinearOperator, aslinearoperator 
from scipy.sparse import csr_matrix, vstack
from scipy.linalg import lstsq, solve
from scipy.sparse.linalg import LinearOperator, cg, lsqr, gmres
from scipy.signal import convolve, filtfilt

In [2]:
client = pylops_distributed.utils.backend.dask(processes=False, threads_per_worker=2,
                                               n_workers=2)
client

Client Scheduler: inproc://10.0.0.10/2723/1 Dashboard: http://localhost:51463/status,Cluster Workers: 2 Cores: 4 Memory: 8.59 GB


### CG for square systems

Let's just try out the solver with a matrix 

In [3]:
n = 10
x = np.ones(n)

A = np.random.randn(n, n)
A = np.dot(A.T, A)
print('eigs', np.linalg.eig(A)[0])

#A = MatrixLinearOperator(A)
Aop = aslinearoperator(A)

y = Aop.matvec(x)
xinv_sp = cg(Aop, y, maxiter=n)[0]
xinv = pylops_distributed.optimization.cg.cg(Aop, y, np.zeros_like(x), n)
print(xinv_sp)
print(xinv)

eigs [37.61103701 29.30501854 17.12882562 14.90751742  6.23435197  5.85914099
  0.11447031  1.57599601  0.99483462  2.67647252]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


And with the LinearOperator

In [4]:
Ada = da.from_array(A, chunks=(n//2, n))
Adaop = pylops_distributed.MatrixMult(Ada, compute=(False, False))

y = Aop * np.ones(n)
yy = Adaop * da.ones(n, chunks=(n//2,))

xinv_sp = cg(Aop, yy, maxiter=n)[0]
xinv = pylops_distributed.optimization.cg.cg(Adaop, yy, da.zeros(n), n)
print(xinv_sp)
print(xinv)
print(xinv.compute())

[1.         1.         1.00000001 1.00000001 1.         1.00000001
 1.         1.         1.         1.        ]
dask.array<add, shape=(10,), dtype=float64, chunksize=(5,)>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [5]:
%timeit -n 3 -r 1 cg(Aop, y, maxiter=n)[0]
%timeit -n 3 -r 1 pylops_distributed.optimization.cg.cg(Aop, y, da.zeros(n), n).compute()
%timeit -n 3 -r 1 pylops_distributed.optimization.cg.cg(Aop, y, da.zeros(n), n, compute=True)

2.77 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


distributed.scheduler - ERROR - Workers don't have promised key: ['inproc://10.0.0.10/2723/3'], ('zeros-b07206f3d61c6132d9f94f96c6947766', 0)
NoneType: None
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('zeros-b07206f3d61c6132d9f94f96c6947766', 0)": ['inproc://10.0.0.10/2723/3']}
distributed.scheduler - ERROR - Workers don't have promised key: ['inproc://10.0.0.10/2723/3'], ('zeros-b07206f3d61c6132d9f94f96c6947766', 0)
NoneType: None
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('zeros-b07206f3d61c6132d9f94f96c6947766', 0)": ['inproc://10.0.0.10/2723/3']}


97.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)
46.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


### CGLS for rectangular systems

In [6]:
n, m = 100, 6
x = np.ones(m)

A = np.random.randn(n, m)
Ada = da.from_array(A, chunks=(n//2, m), name='Aop')
Aop = pylops_distributed.MatrixMult(Ada, compute=(False, False))
y = Aop * da.ones(m, name='x')

xinv = pylops_distributed.optimization.cg.cgls(Aop, y, da.zeros(m, name='x0'), m)

In [7]:
y.visualize(rankdir="LR")
xinv.visualize(rankdir="LR")

In [8]:
print(xinv.compute())

[1. 1. 1. 1. 1. 1.]


In [9]:
%timeit -n 3 -r 1 pylops_distributed.optimization.cg.cgls(Aop, y, da.zeros(m), 20).compute()
%timeit -n 3 -r 1 pylops_distributed.optimization.cg.cgls(Aop, y, da.zeros(m), 20, compute=True)

2.15 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)
12 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)
